In [7]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Generate fake data
# Generate a binary classification dataset using make_classification
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_classes=2, random_state=42)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader for batch processing
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# 2. Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output is 1 because it's binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Instantiate the model, define the loss function and optimizer
input_size = X_train.shape[1]
model = SimpleNN(input_size)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. Train the model
epochs = 100
for epoch in range(epochs):
    running_loss = 0.0
    for x_batch, y_batch in train_loader:
        # Forward pass
        y_pred = model(x_batch).squeeze()  # Remove extra dimension
        loss = criterion(y_pred, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# 4. Evaluate the model on the test set
with torch.no_grad():
    outputs = model(X_test_tensor).squeeze()
    predicted = (outputs > 0.5).float()  # Convert to binary prediction
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f'Accuracy on test set: {accuracy:.4f}')

# Optional: Plot the loss or accuracy if desired


Epoch [1/100], Loss: 0.6885
Epoch [2/100], Loss: 0.6568
Epoch [3/100], Loss: 0.6155
Epoch [4/100], Loss: 0.5647
Epoch [5/100], Loss: 0.5009
Epoch [6/100], Loss: 0.4360
Epoch [7/100], Loss: 0.3856
Epoch [8/100], Loss: 0.3455
Epoch [9/100], Loss: 0.3242
Epoch [10/100], Loss: 0.3078
Epoch [11/100], Loss: 0.2985
Epoch [12/100], Loss: 0.2833
Epoch [13/100], Loss: 0.2702
Epoch [14/100], Loss: 0.2674
Epoch [15/100], Loss: 0.2528
Epoch [16/100], Loss: 0.2457
Epoch [17/100], Loss: 0.2406
Epoch [18/100], Loss: 0.2333
Epoch [19/100], Loss: 0.2253
Epoch [20/100], Loss: 0.2258
Epoch [21/100], Loss: 0.2101
Epoch [22/100], Loss: 0.2054
Epoch [23/100], Loss: 0.2018
Epoch [24/100], Loss: 0.1921
Epoch [25/100], Loss: 0.1876
Epoch [26/100], Loss: 0.1849
Epoch [27/100], Loss: 0.1801
Epoch [28/100], Loss: 0.1753
Epoch [29/100], Loss: 0.1733
Epoch [30/100], Loss: 0.1653
Epoch [31/100], Loss: 0.1601
Epoch [32/100], Loss: 0.1565
Epoch [33/100], Loss: 0.1518
Epoch [34/100], Loss: 0.1481
Epoch [35/100], Loss: 0

In [10]:
model(torch.tensor([1.0 for _ in range(10)]))

tensor([0.0009], grad_fn=<SigmoidBackward0>)

In [3]:
import pandas as pd

In [4]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9
0,1.125100,1.178124,0.493516,0.790880,-0.614278,1.347020,1.419515,1.357325,0.966041,-1.981139
1,-0.564641,3.638629,-1.522415,-1.541705,1.616697,4.781310,3.190292,-0.890254,1.438826,-3.828748
2,0.516313,2.165426,-0.628486,-0.386923,0.492518,1.442381,1.332905,-1.958175,-0.348803,-1.804124
3,0.537282,0.966618,-0.115420,0.670755,-0.958516,0.871440,0.508186,-1.034471,-1.654176,-1.910503
4,0.278385,1.065828,-1.724917,-2.235667,0.715107,0.731249,-0.674119,0.598330,-0.524283,1.047610
...,...,...,...,...,...,...,...,...,...,...
995,-0.906303,-0.527162,-1.511787,-1.697166,-0.585131,0.160046,-2.225249,1.480886,-0.934154,1.151678
996,1.225603,-1.389881,-0.406775,-1.606446,2.500944,-1.089977,0.452517,-1.765429,1.297249,4.705105
997,1.650153,-0.692165,-2.049206,-1.610471,0.119157,-0.876080,-1.368269,-1.302577,-1.285505,3.328569
998,-1.186603,-1.414598,-0.121520,-1.440709,1.630283,-2.034632,-1.537456,-1.421465,-0.028340,3.413932
